# Returning values with `try-catch-finally`

In `scala` even the `try-catch` blocks can return a value.

Let's imagine that we want to do some operation, for example some I/O that could fail. A good habit would be to throw an exception when some instruction could fail, wrap those instructions in a try, handle the exceptions in the catch, and use a finally to do those operations that are needed to be done in any case (for example to close the files, which would stay open as long as the JVM is running).

We will have to handle two exceptions: `NumberFormatException` and `FileNotFoundException`.

First thing let's create the files that we need for the example.

In [30]:
import java.io._

val pw1 = new PrintWriter(new File("/tmp/good.txt"))
pw1.write("200")
pw1.close
val pw2 = new PrintWriter(new File("/tmp/bad.txt"))
pw2.write("zzz")
pw2.close

import java.io._


pw1: PrintWriter = java.io.PrintWriter@6df971af
pw2: PrintWriter = java.io.PrintWriter@6b283f99

Let's have a function `printNumberInfo` accepting a file name. After printing some info about the number in the file, we will close the file.

In [31]:
def printNumberInfo(filename: String) {
    val source = scala.io.Source.fromFile(filename)
    val number = source.getLines.toArray.head.toInt
    
    if (number > 10) {
        println("The number is big")
    } else {
        println("The number is small")
    }
    
    source.close
}

printNumberInfo("/tmp/good.txt")   //Contains a number
printNumberInfo("/tmp/bad.txt")    //Contains a string
printNumberInfo("/tmp/nofile.txt") //Does not exist


The number is big


: 

When we call `printNumberInfo("/tmp/bad.txt")` the exception is preventing the file from being closed. 

Time to use a `try-catch-finally` to handle the exception, print some better info, and close the file. `source` is defined as `Option` to extend its scope over the whole `try-catch-finally`.

In [32]:
import java.io._
import scala.io._

def printNumberInfo(filename: String) {
    var source = None: Option[Source]

    try {
        source = Some(Source.fromFile(filename))
        val number = source.get.getLines.toArray.head.toInt
    
        if (number > 10) {
            println(filename + " -> The number is big")
        } else {
            println(filename + " -> The number is small")
        }
    } catch {
        case ex: NumberFormatException => println(filename + " -> No number found")
        case ex: FileNotFoundException => println(filename + " -> Cannot read the file")
    } finally {
        println("Finally close the file if necessary\n")
        if (source.isDefined) source.get.close 
    }
}

printNumberInfo("/tmp/good.txt")   //Contains a number
printNumberInfo("/tmp/bad.txt")    //Contains a string
printNumberInfo("/tmp/nofile.txt") //Does not exist


/tmp/good.txt -> The number is big
Finally close the file if necessary

/tmp/bad.txt -> No number found
Finally close the file if necessary

/tmp/nofile.txt -> Cannot read the file
Finally close the file if necessary



import java.io._

import scala.io._


defined function printNumberInfo

Very well. This is the canonical use of the `try-catch-finally` as one would do in any language. But this is `scala` and we can do more than that. Let's imagine that we want to return an `Int` representing a status: `0` if everything is OK, any other number if something is wrong.

As one could imagine in `scala` even the `try` returns a value. What I personally wanted to understand originally was: *When I return a value in the `try` or in the `catch`, what happens when I reach the `finally`? 

In [33]:
import java.io._
import scala.io._

def printNumberInfo(filename: String): Int = {
    var source = None: Option[Source]

    val status: Int = try {
        source = Some(Source.fromFile(filename))
        val number = source.get.getLines.toArray.head.toInt
    
        if (number > 10) {
            println(filename + " -> The number is big")
        } else {
            println(filename + " -> The number is small")
        }
        
        0
        
    } catch {
        case ex: NumberFormatException => { 
            println(filename + " -> No number found")
            1
        }
        case ex: FileNotFoundException => {
            println(filename + " -> Cannot read the file")
            2
        }
    } finally {
        println("Finally close the file if necessary\n")
        if (source.isDefined) source.get.close
        23 //This is simply ignored
    }
        status
}

val goodStatus = printNumberInfo("/tmp/good.txt")   //Contains a number
val badStatus = printNumberInfo("/tmp/bad.txt")    //Contains a string
val noFileStatus = printNumberInfo("/tmp/nofile.txt") //Does not exist


/tmp/good.txt -> The number is big
Finally close the file if necessary

/tmp/bad.txt -> No number found
Finally close the file if necessary

/tmp/nofile.txt -> Cannot read the file
Finally close the file if necessary



import java.io._

import scala.io._


defined function printNumberInfo
goodStatus: Int = 0
badStatus: Int = 1
noFileStatus: Int = 2

As you can see the returned values are those defined in the `try` and the `catch`. Despite the returned value in `try` and `catch`, the `finally` is still executed. Even more interesting: if a value is specified in the `finally`, as the `23` in the example, the compiler won't complain but that value will simply be ignored.

Let's play some more. What happens if we use `return`?

In [34]:
import java.io._
import scala.io._

def printNumberInfo(filename: String): Int = {
    var source = None: Option[Source]

    val status: Int = try {
        source = Some(Source.fromFile(filename))
        val number = source.get.getLines.toArray.head.toInt
    
        if (number > 10) {
            println(filename + " -> The number is big")
        } else {
            println(filename + " -> The number is small")
        }
        
        return 0
        
    } catch {
        case ex: NumberFormatException => { 
            println(filename + " -> No number found")
            return 1
        }
        case ex: FileNotFoundException => {
            println(filename + " -> Cannot read the file")
            return 2
        }
    } finally {
        println("Finally close the file if necessary\n")
        if (source.isDefined) source.get.close
        return 23 //This is not ignored when `return` is used
    }
    
    status
}

val goodStatus = printNumberInfo("/tmp/good.txt")   //Contains a number
val badStatus = printNumberInfo("/tmp/bad.txt")    //Contains a string
val noFileStatus = printNumberInfo("/tmp/nofile.txt") //Does not exist


/tmp/good.txt -> The number is big
Finally close the file if necessary

/tmp/bad.txt -> No number found
Finally close the file if necessary

/tmp/nofile.txt -> Cannot read the file
Finally close the file if necessary



import java.io._

import scala.io._


defined function printNumberInfo
goodStatus: Int = 23
badStatus: Int = 23
noFileStatus: Int = 23

The `return` in the `finally` is overriding those in `try` and `catch` bodies. If you don't have any `return` in the `finally` those in the `try` and `catch` will be used.

**Beware**: the `return` will make you exit from the function while just indicating the value will just get you out of the current scope.

## References

A few links:

* https://alvinalexander.com/scala/how-declare-variable-option-before-try-catch-finally-scala
* https://alvinalexander.com/scala/how-to-write-text-files-in-scala-printwriter-filewriter